In [1]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from snntorch import utils
from snntorch import functional as SF
from snntorch import surrogate

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import itertools
import cv2
from tqdm import tqdm
# from collections import defaultdict

from module.custom_data import LoadDataset
from module import custom_data, network, compute_loss, view
from module.const import *

import matplotlib.pyplot as plt
from IPython.display import HTML

from collections import defaultdict


# いろいろ定義

In [2]:

train_dataset = LoadDataset(dataset_dir=DATASET_PATH, dataset_accevent_dir=DATASET_ACCEVENT_PATH, raw_event_dir=RAW_EVENT_PATH, accumulate_time=ACCUMULATE_EVENT_MICROTIME , input_height=INPUT_HEIGHT, input_width=INPUT_WIDTH,train=True)
test_dataset = LoadDataset(dataset_dir=DATASET_PATH, dataset_accevent_dir=DATASET_ACCEVENT_PATH, raw_event_dir=RAW_EVENT_PATH, accumulate_time=ACCUMULATE_EVENT_MICROTIME , input_height=INPUT_HEIGHT, input_width=INPUT_WIDTH, train=False)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_TEST, collate_fn=custom_data.custom_collate, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE_TEST, collate_fn=custom_data.custom_collate, shuffle=False,)



net = NET
net.network.load_state_dict(torch.load(MODEL_PATH))
# corract_rate  = 0.5
events, _ = train_dataset[0]
num_steps = events.shape[0]



# 出力結果の解析
危険とみなすスパイク数の閾値を変化させたときの、出力画像やiouを算出する


In [3]:
# def get_first_frame(video_path):
#     cap = cv2.VideoCapture(video_path)
#     _, frame = cap.read()
#     return frame

# def get_first_events(events):
#     events = events.to('cpu')
#     if BOOL_DISTINGUISH_EVENT:
#         first_events = np.zeros((INPUT_HEIGHT, INPUT_WIDTH, 3)) # bgr
#         first_events[:,:,0] = events[0,0,0] * 255# r 
#         first_events[:,:,1] = events[0,0,1] * 255
#         return first_events
def save_img(number, events, pred_pro, label, image_path):
    # label = label.reshape((pixel, pixel)).to('cpu')
    # print(pred_pro.shape)
    number_str = str(number).zfill(5)
    num_steps = len(pred_pro)
    nrows = 2
    ncols = 5
    fig = plt.figure()
    ax1 = fig.add_subplot(231)
    ax2 = fig.add_subplot(232)
    ax3 = fig.add_subplot(233)
    ax4 = fig.add_subplot(234)
    ax5 = fig.add_subplot(235)
    ax6 = fig.add_subplot(236)


    # dem_filename = f'dem_{str(number).npy}'
    # dem_path = os.path.join(DEM_NP_PATH, dem_filename)
    # dem = np.load(dem_path)
    # ax1.imshow(dem)

    video_filename = f'{number_str}.avi'
    video_path = os.path.join(VIDEO_PATH, video_filename)
    first_frame = view.get_first_frame(video_path)
    ax2.imshow(first_frame)

    first_events = view.get_first_events(events)
    ax3.imshow(first_events)

    label_ =label.reshape((INPUT_HEIGHT, INPUT_WIDTH)).to('cpu')
    ax4.imshow(label_)
    ax4.set_title('label')

    pred_pro_ = pred_pro[:, 1, :, :]
    pred_pro_ = pred_pro_.reshape((INPUT_HEIGHT, INPUT_WIDTH)).to('cpu').detach().numpy().copy()
    ax5.imshow(pred_pro_)
    ax5.set_title('pred_pro')

    ax6.imshow(np.where(pred_pro_>=CORRECT_RATE, 1, 0))
    ax6.set_title('estimate')
    plt.tight_layout()
    fig.savefig(image_path)
    plt.close()

hist = defaultdict(list)
result_dir = 'result_img'
if os.path.exists(result_dir):
        shutil.rmtree(result_dir)
os.makedirs(result_dir)

with torch.no_grad():
    net.network.eval()
    for i, (events, label) in enumerate(iter(test_loader)):
        events = events.to(DEVICE)
        label = label.to(DEVICE)
        batch = len(events[0])
        # print(events.shape)# TBCHW
        # events = events.reshape(num_steps, batch, INPUT_CHANNEL, INPUT_HEIGHT, INPUT_WIDTH)
        pred_pro = net(events)
        
        iou = compute_loss.culc_iou(pred_pro, label, CORRECT_RATE)

        # pred_pro = compute_loss.show_pred(pred_pro, correct_rate)
        img_path = os.path.join(result_dir, f'{str(i).zfill(5)}.png')
      
        save_img(i, events, pred_pro, label, img_path)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

## 閾値毎のIOUの平均算出

In [6]:
for key, value in hist.items():
    print(f'IoU_{key}:{np.mean(value)}')

IoU_0.1:0.7675151172094047
IoU_0.2:0.7658856228366494
IoU_0.3:0.7599267065711319
IoU_0.4:0.7599267065711319
IoU_0.5:0.7599267065711319
IoU_0.6:0.7427386721968651
IoU_0.7:0.7427386721968651
IoU_0.8:0.7220206386968493
IoU_0.9:0.6768500855378806
